# Multy Layer Perceptron

Load MNIST or read the downloaded version:

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import mnist

In [2]:
INPUT_DATA_DIR = '/tmp/tensorflow/mnist/input_data'
data_sets = input_data.read_data_sets(INPUT_DATA_DIR, one_hot=True)

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz


Create the placeholders:

In [3]:
import tensorflow as tf

In [4]:
BATCH_SIZE = 64
FEATURES_NUM = mnist.IMAGE_PIXELS
CLASSES_NUM = mnist.NUM_CLASSES
with tf.variable_scope("dataset"):
    x = tf.placeholder(tf.float32, shape=(BATCH_SIZE, FEATURES_NUM), name='features')
    y = tf.placeholder(tf.float32, shape=(BATCH_SIZE, CLASSES_NUM), name='labels')

Write the initializer for weights and bias

In [5]:
def init_weight(shape, stddev=0.1, name=None):
    init = tf.truncated_normal(shape, stddev=stddev)
    return tf.Variable(init, name=name)

def init_bias(shape, def_value = 0.1, name=None):
    init = tf.constant(def_value, shape=shape)
    return tf.Variable(init, name=name)

def build_mlp_layer(input_shape, output_shape, scope_name, stddev=0.1, def_value=0.1):
    with tf.name_scope(scope_name):
        weight = init_weight((input_shape, output_shape), stddev=stddev, name="weights")
        bias = init_bias((output_shape,), def_value=def_value, name="bias")
        return weight, bias, scope_name

In [6]:
HIDDEN_FEATURES_1 = 256
HIDDEN_FEATURES_2 = 256

layers = [build_mlp_layer(input_shape, output_shape, scope_name) \
          for input_shape, output_shape, scope_name in [(FEATURES_NUM, HIDDEN_FEATURES_1, "Layer_1"),
                                                        (HIDDEN_FEATURES_1, HIDDEN_FEATURES_2, "Layer_2"),
                                                        (HIDDEN_FEATURES_2, CLASSES_NUM, "Output_layer")
]]

Write a method, which builds a simple MLP

In [7]:
def mlp(x, layers):
    result = x
    for weight, bias, scope_name in layers:
        with tf.name_scope(scope_name + "_calculate"):
            result = tf.add(tf.matmul(result, weight), bias)
    return tf.nn.relu(result)

In [8]:
logits = mlp(x, layers)

Create loss

In [9]:
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y))

In [10]:
learning_rate = tf.placeholder(tf.float32)

The place holder for a learning rate

In [11]:
optimizer_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
with tf.name_scope("accuracy_calculation"):
    pred = tf.nn.softmax(logits)
    correct_predictions = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

Add to tensorboard values

In [12]:
for weight, bias, scope_name in layers:
    tf.summary.histogram(scope_name + "_weights", weight)
    tf.summary.histogram(scope_name + "_bias", bias)
    
_ = tf.summary.scalar("loss", loss)
_ = tf.summary.scalar("accuracy", accuracy)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

Run learning

In [13]:
MAX_STEP = 15000
PRINT_INFO_EVERY = 100
TB_PATH = "/tmp/tensorflow/mnist/logs/mlp"

In [14]:
from tqdm import tqdm

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    summary_merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter(TB_PATH, sess.graph)
    
    for step in tqdm(range(MAX_STEP)):
        batch = data_sets.train.next_batch(BATCH_SIZE)
        result = sess.run([optimizer_step, summary_merged], 
                            feed_dict={x: batch[0], y: batch[1], learning_rate: 1e-4})
        writer.add_summary(result[-1], step)
     
    batch = data_sets.train.next_batch(BATCH_SIZE)
    train_acc = accuracy.eval(feed_dict={x: batch[0], y: batch[1]})
    print("Train accuracy: {:0.3f}".format(train_acc))
    
    batch = data_sets.test.next_batch(BATCH_SIZE)
    test_acc = accuracy.eval(feed_dict={x: batch[0], y: batch[1]})
    print("Test accuracy: {:0.3f}".format(test_acc))
    
    batch = data_sets.validation.next_batch(BATCH_SIZE)
    val_acc = accuracy.eval(feed_dict={x: batch[0], y: batch[1]})
    print("Val accuracy: {:0.3f}".format(val_acc))  
    
    writer.close()

100%|██████████| 15000/15000 [03:41<00:00, 67.74it/s]

Train accuracy: 0.609
Test accuracy: 0.562
Val accuracy: 0.547


Now if you run tensorboard --logdir /tmp/tensorflow/mnist/logs/mlp you'll se the statistics of the process